In [13]:
#"""Sample prediction script for TensorFlow 2"""
import argparse
import tensorflow as tf
import numpy as np
import PIL.Image
import os

MODEL_FILENAME = 'model.pb'
LABELS_FILENAME = 'labels.txt'


class ObjectDetection:
    INPUT_TENSOR_NAME = 'image_tensor:0'
    OUTPUT_TENSOR_NAMES = ['detected_boxes:0', 'detected_scores:0', 'detected_classes:0']

    def __init__(self, model_filename):
        graph_def = tf.compat.v1.GraphDef()
        with open(model_filename, 'rb') as f:
            graph_def.ParseFromString(f.read())

        self.graph = tf.Graph()
        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        # Get input shape
        with tf.compat.v1.Session(graph=self.graph) as sess:
            self.input_shape = sess.graph.get_tensor_by_name(self.INPUT_TENSOR_NAME).shape.as_list()[1:3]

    def predict_image(self, image):
        image = image.convert('RGB') if image.mode != 'RGB' else image
        image = image.resize(self.input_shape)

        inputs = np.array(image, dtype=np.float32)[np.newaxis, :, :, :]
        with tf.compat.v1.Session(graph=self.graph) as sess:
            output_tensors = [sess.graph.get_tensor_by_name(n) for n in self.OUTPUT_TENSOR_NAMES]
            outputs = sess.run(output_tensors, {self.INPUT_TENSOR_NAME: inputs})
            return outputs


def predict(model_filename, image_filename):
    od_model = ObjectDetection(model_filename)

    image = PIL.Image.open(image_filename)
    return od_model.predict_image(image)


def main():
    #parser = argparse.ArgumentParser('Object Detection for Custom Vision TensorFlow model')
    #parser.add_argument('image_filename', type=str, help='Filename for the input image')
    #parser.add_argument('--model_filename', type=str, default=MODEL_FILENAME, help='Filename for the tensorflow model')
    #parser.add_argument('--labels_filename', type=str, default=LABELS_FILENAME, help='Filename for the labels file')
    #args = parser.parse_args()

    labels_filename = "labels.txt"
    path = "D:/Styler/Fademeet/outwear/"
 
    #predictions = predict(args.model_filename, args.image_filename)
    for filename in os.listdir(path):
        predictions = predict(MODEL_FILENAME, path+filename)
        with open(labels_filename) as f:
            labels = [l.strip() for l in f.readlines()]

        #count 用來計算圖片裡的服飾件數
        count = 1
        f1 = open(path+"/"+filename.split('.')[0]+".json", "a") 
        f1.write(str("{"+f"'filename':'{filename}',".replace("'",'"').replace("(",'').replace(")",'')))
        f1.close()

        for pred in zip(*predictions):
            print(f"Class: {labels[pred[2]]}, Probability: {pred[1]}, Bounding box: {pred[0]}")
            if(pred[1]>0.5):
                
                # print(filename)
                # print(f"Class{count}: {labels[pred[2]]}, Probability: {pred[1]}, Bounding box: {pred[0]}")
                f1 = open(path+"/"+filename.split('.')[0]+".json", "a") 
                f1.write(str(f"'item{count}':".replace("'",'"')))
                f1.writelines(str("{"+f"'Class':'{labels[pred[2]]}', 'Probability': {pred[1]}, 'Bounding box':[{pred[0][0],pred[0][1],pred[0][2],pred[0][3]}]"+"},").replace("'",'"').replace("(",'').replace(")",''))
                f1.close()
            count = count+1

            #去除掉最夠一個逗號
        with open(path+"/"+filename.split('.')[0]+".json", 'rb+') as f1:
            f1.seek(-1, os.SEEK_END)
            f1.truncate()

        #增加右大括號
        f1 = open(path+"/"+filename.split('.')[0]+".json", "a")
        f1.write("}")
        f1.close()
   

if __name__ == '__main__':
    main()

Class: trousers, Probability: 0.9654560089111328, Bounding box: [0.52166545 0.05582723 0.87756944 0.87954485]
Class: long sleeve top, Probability: 0.2718128561973572, Bounding box: [0.5111536  0.00288956 0.8637743  0.37710553]
Class: trousers, Probability: 0.18883967399597168, Bounding box: [0.20451772 0.05214775 0.8599049  0.9789442 ]
Class: trousers, Probability: 0.08095210790634155, Bounding box: [-0.03550808  0.07748109  0.45297414  0.9523074 ]
Class: trousers, Probability: 0.05592551827430725, Bounding box: [-0.02218679 -0.04378435  0.586566    0.564592  ]
Class: trousers, Probability: 0.04465728998184204, Bounding box: [0.5457187  0.3460415  0.78942734 0.8825837 ]
Class: long sleeve outwear, Probability: 0.025871962308883667, Bounding box: [-0.05907694 -0.01910133  0.63504255  0.2656466 ]
Class: trousers, Probability: 0.021364867687225342, Bounding box: [0.6114024  0.20613784 0.9749756  1.054841  ]
Class: trousers, Probability: 0.01917743682861328, Bounding box: [0.6305988  0.630

Class: trousers, Probability: 0.9886493682861328, Bounding box: [0.3457593  0.347917   0.66919744 0.88273907]
Class: long sleeve top, Probability: 0.906415581703186, Bounding box: [0.2834908  0.0012778  0.6756089  0.51336575]
Class: long sleeve top, Probability: 0.01660245656967163, Bounding box: [-0.06700614 -0.05606422  0.64571357  0.4711382 ]
Class: trousers, Probability: 0.013217538595199585, Bounding box: [0.18864739 0.019806   0.85007644 0.96948373]
Class: trousers, Probability: 0.012431740760803223, Bounding box: [0.10964321 0.37909093 0.5651652  0.9846417 ]
Class: long sleeve top, Probability: 0.010295063257217407, Bounding box: [ 0.36195207 -0.05615266  1.1110941   0.45465785]
Class: trousers, Probability: 0.010173112154006958, Bounding box: [0.4689856  0.7548203  0.67488027 0.9164284 ]
Class: trousers, Probability: 0.8715130090713501, Bounding box: [0.43455118 0.6157044  0.64505655 0.94346225]
Class: long sleeve top, Probability: 0.692719578742981, Bounding box: [0.41483498 0

Class: trousers, Probability: 0.9930866956710815, Bounding box: [0.33059338 0.30484954 0.7172134  0.91897917]
Class: long sleeve top, Probability: 0.9631891846656799, Bounding box: [ 0.28926998 -0.01324339  0.7541681   0.43973154]
Class: trousers, Probability: 0.025045841932296753, Bounding box: [0.22994119 0.03413418 0.7989307  0.97803557]
Class: sling dress, Probability: 0.021079063415527344, Bounding box: [0.7409558 0.3123619 1.0395459 0.9979728]
Class: long sleeve top, Probability: 0.018997550010681152, Bounding box: [-0.05822101 -0.03654324  0.6772648   0.44683474]
Class: sling dress, Probability: 0.018833041191101074, Bounding box: [-0.0350181   0.18799537  0.25250095  1.017761  ]
Class: trousers, Probability: 0.01666077971458435, Bounding box: [0.19310904 0.35181627 0.38561076 0.56239897]
Class: trousers, Probability: 0.013180971145629883, Bounding box: [0.17807145 0.29781714 0.37617838 0.495437  ]
Class: trousers, Probability: 0.012674331665039062, Bounding box: [0.18418491 0.3

Class: long sleeve outwear, Probability: 0.9293835163116455, Bounding box: [0.3160319  0.02393308 0.6834326  0.59748936]
Class: trousers, Probability: 0.2209770381450653, Bounding box: [0.3490953  0.26036125 0.57180333 0.9145929 ]
Class: long sleeve outwear, Probability: 0.019407957792282104, Bounding box: [0.34986788 0.01934245 0.6971033  0.30547634]
Class: short sleeve outwear, Probability: 0.01940023899078369, Bounding box: [0.34525558 0.22316036 0.63300455 0.6593617 ]
Class: trousers, Probability: 0.01881900429725647, Bounding box: [0.45385027 0.6267325  0.5909997  0.8876515 ]
Class: trousers, Probability: 0.015057563781738281, Bounding box: [0.15200958 0.51098764 0.34647495 0.96081626]
Class: short sleeve dress, Probability: 0.013975977897644043, Bounding box: [0.30634105 0.2204706  0.6899691  1.0467911 ]
Class: shorts, Probability: 0.011188745498657227, Bounding box: [0.42930833 0.25432265 0.6150604  0.5133373 ]
Class: sling, Probability: 0.010674864053726196, Bounding box: [-0.0

Class: trousers, Probability: 0.9707342386245728, Bounding box: [0.36516577 0.26374027 0.62685835 0.9007504 ]
Class: long sleeve top, Probability: 0.7911105155944824, Bounding box: [ 0.3273055  -0.00451587  0.7081916   0.40618742]
Class: long sleeve top, Probability: 0.033158689737319946, Bounding box: [-0.05002157 -0.01384323  0.29072636  0.16002262]
Class: skirt, Probability: 0.025777041912078857, Bounding box: [-0.02722277  0.33963013  0.3106252   1.0289369 ]
Class: sling dress, Probability: 0.020484745502471924, Bounding box: [0.8425027  0.30673268 0.99703443 0.5147716 ]
Class: long sleeve top, Probability: 0.014562666416168213, Bounding box: [ 0.361563   -0.04228477  1.1203966   0.42388284]
Class: long sleeve top, Probability: 0.014037996530532837, Bounding box: [-0.11127052 -0.03963521  0.63036335  0.4284517 ]
Class: trousers, Probability: 0.01345604658126831, Bounding box: [0.9305841  0.40511706 1.0034702  0.54819465]
Class: short sleeve top, Probability: 0.012544423341751099, B

...

Class: trousers, Probability: 0.9335851669311523, Bounding box: [0.36197025 0.33276325 0.639996   0.9389923 ]
Class: long sleeve outwear, Probability: 0.9150846004486084, Bounding box: [ 0.33430815 -0.0104949   0.6897223   0.45931286]
Class: shorts, Probability: 0.04585900902748108, Bounding box: [0.41537088 0.67139703 0.6126117  0.9226373 ]
Class: long sleeve outwear, Probability: 0.04012289643287659, Bounding box: [0.35403588 0.21710771 0.66409886 0.67775357]
Class: long sleeve outwear, Probability: 0.037618428468704224, Bounding box: [0.27683848 0.0916115  0.7197457  0.94579375]
Class: shorts, Probability: 0.028579652309417725, Bounding box: [0.37737563 0.28718957 0.5828568  0.47393414]
Class: long sleeve outwear, Probability: 0.025238007307052612, Bounding box: [0.44158024 0.12054005 1.0341196  0.8906286 ]
Class: long sleeve outwear, Probability: 0.020748794078826904, Bounding box: [ 0.46737453 -0.10857253  1.0993598   0.48013544]
Class: long sleeve outwear, Probability: 0.01964050

Class: trousers, Probability: 0.996515154838562, Bounding box: [0.35107097 0.35184368 0.6702908  0.99981725]
Class: long sleeve outwear, Probability: 0.9681143760681152, Bounding box: [ 0.2690432  -0.00516596  0.7376286   0.55680895]
Class: trousers, Probability: 0.03474709391593933, Bounding box: [0.09646063 0.38536832 0.5838305  1.0104274 ]
Class: long sleeve outwear, Probability: 0.02405056357383728, Bounding box: [0.31625596 0.2467957  0.7140757  0.6070813 ]
Class: trousers, Probability: 0.019064903259277344, Bounding box: [-0.01707223  0.13402054  0.24736267  0.8756243 ]
Class: trousers, Probability: 0.014490485191345215, Bounding box: [0.14702237 0.1774618  0.5129615  0.88280016]
Class: long sleeve outwear, Probability: 0.013354659080505371, Bounding box: [0.26617226 0.00973263 0.50719106 0.5043634 ]
Class: trousers, Probability: 0.012813985347747803, Bounding box: [-0.01629281  0.4134568   0.28932953  1.0006907 ]
Class: long sleeve top, Probability: 0.012155473232269287, Boundin

Class: trousers, Probability: 0.9856654405593872, Bounding box: [0.33625498 0.28827813 0.6551328  0.8736266 ]
Class: long sleeve outwear, Probability: 0.9627819657325745, Bounding box: [ 0.29682726 -0.00644349  0.71790725  0.43675268]
Class: long sleeve outwear, Probability: 0.024741023778915405, Bounding box: [0.21568197 0.01863241 0.8171572  0.95989424]
Class: trousers, Probability: 0.021112024784088135, Bounding box: [0.06967315 0.40819657 0.6455443  0.98862267]
Class: trousers, Probability: 0.019764065742492676, Bounding box: [0.29401487 0.4449393  0.68491125 1.0582547 ]
Class: shorts, Probability: 0.014204710721969604, Bounding box: [0.40370524 0.23674445 0.6230097  0.40515083]
Class: shorts, Probability: 0.013617336750030518, Bounding box: [0.40576017 0.32305655 0.6185894  0.5273529 ]
Class: shorts, Probability: 0.01317790150642395, Bounding box: [0.37104115 0.16243818 0.57230026 0.37411764]
Class: long sleeve outwear, Probability: 0.013035297393798828, Bounding box: [-0.07212743

Class: trousers, Probability: 0.9063101410865784, Bounding box: [0.39078748 0.32476574 0.6651375  0.89110917]
Class: long sleeve top, Probability: 0.6845903396606445, Bounding box: [ 0.34037936 -0.01019624  0.70745623  0.46943375]
Class: long sleeve top, Probability: 0.024601250886917114, Bounding box: [-0.10685721 -0.04767086  0.63206124  0.47097808]
Class: trousers, Probability: 0.020180583000183105, Bounding box: [0.40065435 0.62947774 0.57597536 0.8828217 ]
Class: trousers, Probability: 0.01654791831970215, Bounding box: [0.46731356 0.6050615  0.6278831  0.86203426]
Class: long sleeve outwear, Probability: 0.014383912086486816, Bounding box: [0.13475731 0.01181868 0.8995997  0.9801512 ]
Class: long sleeve outwear, Probability: 0.013305425643920898, Bounding box: [-0.04847777  0.20182344  1.0421517   0.80754936]
Class: trousers, Probability: 0.010315179824829102, Bounding box: [0.53797907 0.59078074 0.61543506 0.85286593]
Class: long sleeve top, Probability: 0.010022073984146118, Bo

Class: trousers, Probability: 0.9860166311264038, Bounding box: [0.42164275 0.54805136 0.61533463 0.96460736]
Class: long sleeve outwear, Probability: 0.611960232257843, Bounding box: [0.39315397 0.3375476  0.6329194  0.65045965]
Class: skirt, Probability: 0.05676338076591492, Bounding box: [-0.02706766 -0.03573931  0.99375176  0.28391993]
Class: trousers, Probability: 0.04844734072685242, Bounding box: [0.44559956 0.57751155 0.5945927  0.8412323 ]
Class: long sleeve outwear, Probability: 0.03752109408378601, Bounding box: [0.03975588 0.31370777 0.6212141  0.6699609 ]
Class: shorts, Probability: 0.02016615867614746, Bounding box: [0.401674   0.44334343 0.57230604 0.6652808 ]
Class: long sleeve outwear, Probability: 0.02007192373275757, Bounding box: [0.30374455 0.2898985  0.694637   0.69259846]
Class: trousers, Probability: 0.01901555061340332, Bounding box: [0.47797388 0.68141276 0.633102   0.9551845 ]
Class: trousers, Probability: 0.017487555742263794, Bounding box: [0.42551735 0.537